In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from pathlib import Path

from tqdm import tqdm
import numpy as np
import pandas as pd
import dataframe_image as dfi
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle

import src.visualization.visualize as visualize

In [2]:
PROCESSED_DATA_FOLDER_PATH = Path("../data/processed")


## Load data

In [ ]:
all_land_df = pd.read_parquet(PROCESSED_DATA_FOLDER_PATH.joinpath("CWatM_data", "all_land.parquet"))
forcings_land_df = pd.read_parquet(PROCESSED_DATA_FOLDER_PATH.joinpath("CWatM_data", "forcings_land.parquet"))
outputs_land_df = pd.read_parquet(PROCESSED_DATA_FOLDER_PATH.joinpath("CWatM_data", "outputs_land.parquet"))

data_df = pd.concat((all_land_df, forcings_land_df, outputs_land_df), axis=1)
data_df

In [4]:
INPUTS_COLUMNS = list(all_land_df.columns)
FORCINGS_COLUMNS = list(forcings_land_df.columns)
OUTPUTS_COLUMNS = list(outputs_land_df.columns)

## Explore "chanleng - potevap" relation

<div>
<img src="../reports/figures/CWatM_data/scatterplots/chanleng_potevap.png" width="1000"/>
</div>

In [ ]:
input_col = "chanleng"
output_col = "potevap"

marker_style = MarkerStyle(marker=".",
                           fillstyle="full")
fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

axis.scatter(x=data_df[input_col], y=data_df[output_col],
             marker=marker_style,
             s=20,
             edgecolor="none",
             alpha=0.25,
             )

X = data_df[input_col].copy().to_numpy()
X.sort()

def line1(x_array):
    return 1/450000 * (x_array - 10000)**2

def line2(x_array):
    return 1/1200000 * (x_array - 10000)**2

def line3(x_array):
    return 1/53000000000 * (x_array - 20000)**3

axis.plot(X, line1(X), c="red")
axis.plot(X, line2(X), c="orange")
axis.plot(X, line3(X), c="green")

vline1 = 30000
vline2 = 40000
vline3 = 58000

axis.vlines(vline1, -100, 3400, color="red")
axis.vlines(vline2, -100, 3400, color="orange")
axis.vlines(vline3, -100, 3400, color="green")

axis.set_xlim([7000, 80000])
axis.set_ylim([-100, 3400])

axis.set_xlabel(input_col)
axis.set_ylabel(output_col)

fig.tight_layout()
fig.show()

In [6]:
temp_df = data_df.copy()

region_1 = temp_df[(temp_df["chanleng"] < vline1) &
                   (temp_df["potevap"] > line1(temp_df["chanleng"]))]
temp_df = temp_df.loc[list(set(temp_df.index).difference(set(region_1.index)))].copy()

region_2 = temp_df[(temp_df["chanleng"] < vline2) &
                   (temp_df["potevap"] > line2(temp_df["chanleng"]))]
temp_df = temp_df.loc[list(set(temp_df.index).difference(set(region_2.index)))].copy()

region_3 = temp_df[(temp_df["chanleng"] < vline3) &
                   (temp_df["potevap"] > line3(temp_df["chanleng"]))]
temp_df = temp_df.loc[list(set(temp_df.index).difference(set(region_3.index)))].copy()

region_4 = temp_df

####

region_1["region"] = 1
region_2["region"] = 2
region_3["region"] = 3
region_4["region"] = 4

all_regions_df = pd.concat([region_1, region_2, region_3, region_4], axis=0)
regions_df = all_regions_df["region"].copy().to_frame()


## Visualize the four regions

### On a scatterplot

In [ ]:

regions = regions_df["region"].unique()

colors = ['red', 'blue', 'green', 'purple']
marker_style = MarkerStyle(marker=".",
                           fillstyle="full")

fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

for i, region in enumerate(regions):

    # ensure regions_df has same indexes as data_df
    region_indices = regions_df[regions_df["region"] == region].index
    region_indices = set(region_indices).intersection(data_df.index)

    region_data_df = data_df.loc[list(region_indices)]

    axis.scatter(x=region_data_df[input_col], y=region_data_df[output_col],
                    label=f"Region {region}",
                    c=colors[i],
                    marker=marker_style,
                    s=10,
                    edgecolor="none",
                    alpha=0.25,
                    )
    
    region_data_df.sort_values(input_col, ascending=True, inplace=True)

    import matplotlib.patheffects as mpe
    outline = mpe.withStroke(linewidth=4, foreground='white')

    axis.plot(region_data_df[input_col], region_data_df[output_col].rolling(window=3000,
                                                                            # win_type="gaussian",
                                                                            center=True,
                                                                            ).mean(
                                                                                # std=2000
                                                                            ),
                c=colors[i],
                path_effects=[outline],
                label=f"_{region}"
                )

axis.set_xlabel(input_col)
axis.set_ylabel(output_col)
axis.label_outer()

fig.tight_layout()
fig.subplots_adjust(bottom=0.13)
legend = fig.legend(#labels=regions,
                    markerscale=3,
                    loc="lower center",
                    ncol=4)

for legobj in legend.legend_handles:
    legobj.set_alpha(1)
fig.show()


fig.savefig(f"../reports/figures/CwatM_data/scatterplots_regions_chanleng/{input_col}_{output_col}", dpi=300)

### On a world map

In [ ]:
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle

fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Add map features
ax.coastlines()
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')

marker_style = MarkerStyle(marker=".",
                           fillstyle="full")

alpha = 0.5
size = 30

# Plot data points
ax.scatter(region_1.reset_index()['lon'], region_1.reset_index()['lat'],
           color='red', label='Region 1', 
           alpha=alpha, transform=ccrs.PlateCarree(),
           marker=marker_style, s=size, edgecolor="none",
           )
ax.scatter(region_2.reset_index()['lon'], region_2.reset_index()['lat'],
           color='blue', label='Region 2', 
           alpha=alpha, transform=ccrs.PlateCarree(),
           marker=marker_style, s=size, edgecolor="none",
           )
ax.scatter(region_3.reset_index()['lon'], region_3.reset_index()['lat'],
           color='green', label='Region 3', 
           alpha=alpha, transform=ccrs.PlateCarree(),
           marker=marker_style, s=size, edgecolor="none",
           )
ax.scatter(region_4.reset_index()['lon'], region_4.reset_index()['lat'],
           color='purple', label='Region 4', 
           alpha=alpha, transform=ccrs.PlateCarree(),
           marker=marker_style, s=size, edgecolor="none",
           )

plt.show()


In [ ]:

# Create a figure with 2x2 subplots, each with the PlateCarree projection
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 10),
                         subplot_kw={'projection': ccrs.PlateCarree()})

# Flatten the axes array for easy iteration
axes = axes.flatten()

# List of regions, colors, and labels
regions = [region_1, region_2, region_3, region_4]
colors = ['red', 'blue', 'green', 'purple']
labels = ['Region 1', 'Region 2', 'Region 3', 'Region 4']

# Marker style
marker_style = MarkerStyle(marker=".", fillstyle="full")
alpha = 0.25
size = 3

# Plot each region in its own subplot
for ax, region, color, label in zip(axes, regions, colors, labels):
    # Add map features
    ax.coastlines()
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    
    # Plot data points
    ax.scatter(region.reset_index()['lon'], region.reset_index()['lat'],
               color=color, label=label, 
               alpha=alpha, transform=ccrs.PlateCarree(),
               marker=marker_style, s=size, edgecolor="none")
    
    # Set title and legend
    ax.set_title(label)
    ax.legend(loc='lower left')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()

## Make scatterplots with the new regions

In [10]:
from itertools import product
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle

def display_individual_scatterplots(df: pd.DataFrame,
                                    dst_path: Path,
                                    valid_x,
                                    valid_y,
                                    regions_df = None,
                                    regions_2x2 = True
                                    ):

    marker_style = MarkerStyle(marker=".",
                               fillstyle="full")

    combinations = product(valid_x, valid_y)

    for input_col, output_col in tqdm(list(combinations), desc="Computing input-output combinations"):

        # Assuming the DataFrame has 'x' and 'y' columns for the scatter plot
        if regions_df is None:

            fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
            fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

            axis.scatter(x=df[input_col], y=df[output_col],
                         marker=marker_style,
                         edgecolor="none",
                         s=30,
                         alpha=0.25,
                         )
            
            axis.set_xlabel(input_col)
            axis.set_ylabel(output_col)
        else:
            
            palette = {1: "red",
                       2: "blue",
                       3: "green",
                       4: "purple"}
            
            regions = regions_df["region"].unique()

            if regions_2x2:
                fig, axis = plt.subplots(nrows=2, ncols=2, figsize=(8, 6),
                                        sharex=True, sharey=True, constrained_layout=False)
                axis = axis.flatten()
                fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

                for i, region in enumerate(regions):

                    # ensure regions_df has same indexes as data_df
                    region_indices = regions_df[regions_df["region"] == region].index
                    region_indices = set(region_indices).intersection(data_df.index)

                    region_data_df = data_df.loc[list(region_indices)]
                
                    axis[i].scatter(x=region_data_df[input_col], y=region_data_df[output_col],
                                    label=region,
                                    c=palette[region],
                                    marker=marker_style,
                                    s=10,
                                    edgecolor="none",
                                    alpha=0.25,
                                    )
                    
                    
                    axis[i].set_xlabel(input_col)
                    axis[i].set_ylabel(output_col)
                    axis[i].label_outer()
            else:
                fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
                fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

                for i, region in enumerate(regions):

                    # ensure regions_df has same indexes as data_df
                    region_indices = regions_df[regions_df["region"] == region].index
                    region_indices = set(region_indices).intersection(data_df.index)

                    region_data_df = data_df.loc[list(region_indices)]
                
                    axis.scatter(x=region_data_df[input_col], y=region_data_df[output_col],
                                 label=region,
                                 c=palette[region],
                                 marker=marker_style,
                                 s=10,
                                 edgecolor="none",
                                 alpha=0.25,
                                 )
                    
                    region_data_df.sort_values(input_col, ascending=True, inplace=True)

                    import matplotlib.patheffects as mpe
                    outline = mpe.withStroke(linewidth=4, foreground='white')

                    axis.plot(region_data_df[input_col], region_data_df[output_col].rolling(window=3000,
                                                                                            # win_type="gaussian",
                                                                                            center=True,
                                                                                            ).mean(
                                                                                                # std=2000
                                                                                            ),
                              c=palette[region],
                              path_effects=[outline],
                              label=f"_{region}"
                              )
                
                axis.set_xlabel(input_col)
                axis.set_ylabel(output_col)
                axis.label_outer()

        # Adjust layout and display the plots
        fig.tight_layout()
        if regions_df is not None:
            fig.subplots_adjust(bottom=0.13)
            legend = fig.legend(#labels=regions,
                                markerscale=3,
                                loc="lower center",
                                ncol=4)
            
            for legobj in legend.legend_handles:
                legobj.set_alpha(1)

        fig.savefig(dst_path.joinpath(f"{input_col}_{output_col}.png"), dpi=300)

        plt.close()


In [ ]:
display_individual_scatterplots(df=data_df,
                                dst_path=Path("../reports/figures/CWatM_data/scatterplots_regions_chanleng"),
                                valid_x=INPUTS_COLUMNS + FORCINGS_COLUMNS,
                                valid_y=OUTPUTS_COLUMNS,
                                regions_df=regions_df,
                                regions_2x2=False
                                )

In [ ]:
display_individual_scatterplots(df=data_df,
                                dst_path=Path("../reports/figures/CWatM_data/scatterplots_regions_2x2_chanleng"),
                                valid_x=INPUTS_COLUMNS + FORCINGS_COLUMNS,
                                valid_y=OUTPUTS_COLUMNS,
                                regions_df=regions_df,
                                regions_2x2=True
                                )

## Compute the measures based on the new regions

In [ ]:
from src.dependence_measures.compare import compute_bivariate_scores

INPUTS_COLUMNS = ["chanleng"]#list(all_land_df.columns)
FORCINGS_COLUMNS = []#list(forcings_land_df.columns)
OUTPUTS_COLUMNS = list(outputs_land_df.columns)

regions = regions_df["region"].unique()

for region in regions:

    region_indices = regions_df[regions_df["region"] == region].index
    region_indices = set(region_indices).intersection(data_df.index)

    print(region, len(region_indices))

    region_data_df = data_df.loc[list(region_indices)]

    for inputs_columns_split in np.array_split(INPUTS_COLUMNS, 25):

        for forcings_columns_split in np.array_split(FORCINGS_COLUMNS, 2):

            input_cols = inputs_columns_split.tolist() + forcings_columns_split.tolist()

            scores_df = compute_bivariate_scores(region_data_df,
                                                 input_cols=input_cols,
                                                 output_cols=OUTPUTS_COLUMNS,
                                                 dst_file_path=PROCESSED_DATA_FOLDER_PATH.joinpath("bivariate_metrics",
                                                                                                   "CWatM",
                                                                                                   f"measures_chanleng_{region}.csv"),
                                                 return_all=True)


## Use a classifier to understand these regions

In [12]:
temp_df = all_regions_df.copy()

y = temp_df.pop("region").values

temp_df = temp_df.reset_index()
temp_df.pop("chanleng")
X = temp_df

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_cls = RandomForestClassifier(n_estimators=40,
                                max_depth=10,
                                min_samples_split=20,
                                min_samples_leaf=5,
                                max_features=0.25,
                                random_state=23,
                                verbose=2,
                                )

rf_cls.fit(X, y)

In [ ]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_true=y,
                          y_score=rf_cls.predict_proba(X), multi_class="ovr")

auc_score

In [15]:
# pd.DataFrame(data=rf_cls.feature_importances_,
#              index=rf_cls.feature_names_in_).sort_values(by=0, ascending=True).tail(15).plot.barh(legend=False)

In [ ]:
all_regions_df.groupby("region")[["ups", "chanman"]].median()

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(estimator=rf_cls,
                                         X=X,
                                         y=y,
                                         n_repeats=10)

In [ ]:
pd.DataFrame(data=perm_importance.importances_mean,
             index=rf_cls.feature_names_in_).sort_values(by=0, ascending=True).tail(15).plot.barh(legend=False)

In [ ]:
sorted_importances_idx = perm_importance.importances_mean.argsort()[-10:]

importances = pd.DataFrame(
    perm_importance.importances[sorted_importances_idx].T,
    columns=rf_cls.feature_names_in_[sorted_importances_idx],
)

ax = importances.plot.box(vert=False, whis=10, figsize=(8, 6))
ax.set_title("Permutation Feature Importances", fontsize=16)
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in score", fontsize=12)
ax.figure.tight_layout()

fig = ax.get_figure()
fig.savefig(f"../reports/figures/CWatM_data/plot_understanding_chanleng_potevap_permutation_feature_importance.png", dpi=300)

## Make plot to understand the four relationships

In [10]:
# input_col = "ups"
# output_col = "potevap"

# regions = regions_df["region"].unique()

# colors = ['red', 'blue', 'green', 'purple']
# marker_style = MarkerStyle(marker=".",
#                            fillstyle="full")

# fig, axis = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
# fig.suptitle(f"Input '{input_col}' - Output '{output_col}'")

# for i, region in enumerate(regions):

#     # ensure regions_df has same indexes as data_df
#     region_indices = regions_df[regions_df["region"] == region].index
#     region_indices = set(region_indices).intersection(data_df.index)

#     region_data_df = data_df.loc[list(region_indices)]

#     axis.scatter(x=region_data_df[input_col], y=region_data_df[output_col],
#                     label=f"Region {region}",
#                     c=colors[i],
#                     marker=marker_style,
#                     s=10,
#                     edgecolor="none",
#                     alpha=0.25,
#                     )
    
#     region_data_df.sort_values(input_col, ascending=True, inplace=True)

#     import matplotlib.patheffects as mpe
#     outline = mpe.withStroke(linewidth=4, foreground='white')

#     axis.plot(region_data_df[input_col], region_data_df[output_col].rolling(window=3000,
#                                                                             # win_type="gaussian",
#                                                                             center=True,
#                                                                             ).mean(
#                                                                                 # std=2000
#                                                                             ),
#                 c=colors[i],
#                 path_effects=[outline],
#                 label=f"_{region}"
#                 )

# axis.set_xlabel(input_col)
# axis.set_ylabel(output_col)
# axis.label_outer()

# axis.set_xlim([-100, 30000])

# fig.tight_layout()
# fig.subplots_adjust(bottom=0.13)
# legend = fig.legend(#labels=regions,
#                     markerscale=3,
#                     loc="lower center",
#                     ncol=4)

# for legobj in legend.legend_handles:
#     legobj.set_alpha(1)
# fig.show()

In [ ]:
all_regions_df

In [131]:
from matplotlib.markers import MarkerStyle
from matplotlib.colors import TwoSlopeNorm
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg


def plot_scatter_colorscale(data_df: pd.DataFrame,
                            input_col: str,
                            output_col: str,
                            color_col: str,
                            norm,
                            fig = None,
                            axes = None,
                            ):
    
    temp_df = data_df[[input_col] + [output_col] + [color_col]].copy()

    marker_style = MarkerStyle(marker=".",
                               fillstyle="full")

    if fig is None or axes is None:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8,6))

    scatter = axes.scatter(x=temp_df[input_col], y=temp_df[output_col],
                           c=temp_df[color_col],
                           cmap="Reds",
                           norm=norm,
                           marker=marker_style,
                           s=10,
                           edgecolor="none",
                          #  alpha=0.25,
                           )
    
    axes.set_xlabel(input_col)
    axes.set_ylabel(output_col)
    axes.set_title(f"'{color_col}' relation to '{input_col}' - '{output_col}'")

    # Use make_axes_locatable to create a new axis next to your existing axis
    divider = make_axes_locatable(axes)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    # Add colorbar to the new axis
    cbar = fig.colorbar(scatter, cax=cax)
    cbar.set_label(f"{color_col} value", rotation=90, loc="top")
    # cbar.ax.xaxis.set_label_coords(x=0.5, y=2.05, transform=cax.transAxes)

    # cax.text(2, 1.02, f"{color_col} \nvalue", transform=cax.transAxes,
    #          va='center', ha='center')


def plot_scatter_ldd_regions(data_df: pd.DataFrame,
                             fig = None,
                             axes = None,
                             ):
    
    regions_df = data_df["ldd"].isin([1, 3, 7, 9]).to_frame()
    regions_df.columns = ["region"]
    regions_df["region"] = regions_df["region"].map({True: "'ldd' Corner",
                                                     False: "'ldd' Non-corner"})

    regions = regions_df["region"].unique()

    colors = ['purple', 'green', 'green', 'purple']
    marker_style = MarkerStyle(marker=".",
                            fillstyle="full")

    if fig is None or axes is None:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))

    for i, region in enumerate(regions):

        # ensure regions_df has same indexes as data_df
        region_indices = regions_df[regions_df["region"] == region].index
        region_indices = set(region_indices).intersection(data_df.index)

        region_data_df = data_df.loc[list(region_indices)]

        axes.scatter(x=region_data_df["chanleng"], y=region_data_df["potevap"],
                        label=f"Region {region}",
                        c=colors[i],
                        marker=marker_style,
                        s=10,
                        edgecolor="none",
                        alpha=0.25,
                        )
    
    axes.set_xlabel(input_col)
    axes.set_ylabel(output_col)
    axes.set_title(f"'ldd' relation to '{input_col}' - '{output_col}'")
    axes.label_outer()

    # legend = axes.legend(#labels=regions,
    #                     markerscale=3,
    #                     loc="upper right",
    #                     ncol=1)

    # for legobj in legend.legend_handles:
    #     legobj.set_alpha(1)


def plot_ldd_code_matrix(axes):

    ldd_data = np.array([
        [7, 8, 9],
        [4, 5, 6],
        [1, 2, 3]
    ])
    # Define which values should be green (7, 9, 1, 3)
    green_values = {1, 3, 7, 9}
    mask = np.isin(ldd_data, list(green_values))

    # We'll create a simple binary array: 0 for green group, 1 for purple group
    color_data = np.where(mask, 0, 1)

    # Create a ListedColormap: index 0 will map to green, 1 will map to purple
    cmap = mcolors.ListedColormap(['green', 'purple'])

    # Create a normalization so that 0 -> green, 1 -> purple
    norm = mcolors.BoundaryNorm([-0.5, 0.5, 1.5], cmap.N)

    im = axes.imshow(color_data, cmap=cmap, norm=norm)

    # Add the text annotations for each cell
    for i in range(ldd_data.shape[0]):
        for j in range(ldd_data.shape[1]):
            axes.text(j, i, str(ldd_data[i, j]),
                        ha='center', va='center',
                        color='white',
                        weight='bold'
                        )

    # Optionally, remove ticks to have a cleaner look
    axes.set_xticks([])
    axes.set_yticks([])
    axes.set_title("'ldd' code", fontsize=13, y=-0.38)


def plot_understanding_chanleng_potevap_relationships(data_df: pd.DataFrame):

    # fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6),
    #                          sharey=False
    #                          )
    
    fig = plt.figure(figsize=(12, 8))
    
    # Create a GridSpec with 2 rows and 2 columns
    # The top row (2 subplots) will have more space, the bottom row less.
    gs = GridSpec(nrows=2, ncols=2,
                  height_ratios=[0.6, 0.4], 
                #   hspace=0.05,
                  figure=fig)
    
    # Upper subplots
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])

    plot_scatter_colorscale(data_df=data_df,
                            input_col="chanleng",
                            output_col="potevap",
                            color_col="ups",
                            # Set the normalization so that 0 is at the center of the colormap
                            norm = TwoSlopeNorm(vmin=all_regions_df["ups"].min(),
                                                vcenter=6000,
                                                vmax=all_regions_df["ups"].max()
                                                ),
                            fig=fig,
                            axes=ax1
                            )
    
    plot_scatter_ldd_regions(data_df=data_df,
                             fig=fig,
                             axes=ax2
                             )
    ax2.tick_params(axis='x', labelbottom=True)
    ax2.set_xlabel("chanleng")

    # Bottom row for the image (spanning both columns)
    ax_img = fig.add_subplot(gs[1, :])
    
    # Read and plot the image
    img = mpimg.imread("elevation_flowaccu.jpg")
    ax_img.imshow(img)
    ax_img.set_title("CWatM process to determine flow accumulation from elevation", fontsize=16)  # Add a title above the image
    ax_img.axis('off')  # Hide axes for a cleaner look

    gs_ldd = GridSpec(nrows=1, ncols=1,
                      figure=fig,
                      left=0.55, right=0.65,
                      bottom=0.79, top=0.885)
    
    ax_ldd = fig.add_subplot(gs_ldd[0])
    
    plot_ldd_code_matrix(axes=ax_ldd)

    fig.suptitle("Understanding 'chanleng' - 'potevap' patterns", fontsize=22, y=0.995)
    fig.subplots_adjust(top=0.88)
    fig.tight_layout()

    return fig


In [ ]:
# plot_scatter_colorscale(all_regions_df,
#                         input_col="chanleng",
#                         output_col="potevap",
#                         color_col="ups",
#                         # Set the normalization so that 0 is at the center of the colormap
#                         norm = TwoSlopeNorm(vmin=all_regions_df["ups"].min(),
#                                             vcenter=6000,
#                                             vmax=all_regions_df["ups"].max()
#                                             )
# )

# plot_scatter_ldd_regions(all_regions_df)

fig = plot_understanding_chanleng_potevap_relationships(data_df=all_regions_df)

fig.savefig(f"../reports/figures/CWatM_data/plot_understanding_chanleng_potevap.png", dpi=300)
